In [1]:
from unsloth import FastLanguageModel, is_bf16_supported
import torch

max_seq_length = 4096
lora_rank = 64
dtype = None

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./models/Qwen2.5-0.5B",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    dtype = dtype
    #fast_inference = True,
    #max_lora_rank = lora_rank,
    #gpu_memory_utilization = 0.5
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
    use_rslora = False,
    loftq_config = None    
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/anthony/miniconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.2: Fast Qwen2 patching. Transformers: 4.52.4.
   \\   /|    NVIDIA RTX 4500 Ada Generation. Num GPUs = 1. Max memory: 23.994 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
./models/Qwen2.5-0.5B does not have a padding token! Will use pad_token = <|vision_pad|>.


Unsloth 2025.6.2 patched 24 layers with 24 QKV layers, 24 O layers and 24 MLP layers.


In [ ]:
import json

with open("./conversational_faq/openline-questions/conv_sim_derived_questions_faq_33_0_to_7.json", "r") as f:
    conv_data = json.load(f)

conv_data[0]['messages']
context = conv_data[0]['context']
context = conv_data[0]['context']

[{'role': 'user',
  'content': '¿Qué pasos debe seguir un estudiante para convalidar sus prácticas pre profesionales?',
  'context': '\n\nConvalidar Practicas Pre Profesionales\n¿Cuál es el proceso que debe seguir un estudiante para convalidar sus practicas pre pre profesionales?\nLos estudiantes que deseen realizar convalidar sus practicas pre pre profesionales deben ponerse en contacto con el departamento de su Escuela Profesional correspondiente para obtener más información. A continuación se detallan las direcciones de correo electrónico según la escuela profesional:\nFísica, Química o Ciencia de la Computación: escuelas_fc1@uni.edu.pe\nMatemáticas o Ingeniería Física: escuelas_fc2@uni.edu.pe\n\nTraslado Interno\n            TRASLADO INTERNO\n        (del Reglamento de Matrícula Aprobado R.R. N° 0570 del 29.03.22)\n\nArt. 16° De la convalidación de Asignaturas y su aplicación\n\nLa Convalidación de Asignatura es el acto académico y administrativo mediante el cual la UNI a través de

In [ ]:
## Contextual Etiqueta
new_tokens = ["<|im_start|>context", "<|context|>"]
tokenizer.add_tokens(new_tokens)
model.resize_token_embeddings(len(tokenizer))
tokenizer.chat_template = """{% for message in messages %}
{% if message['role'] == 'system' %}
<|im_start|>system
{{ message['content'] }}<|im_end|>
{% elif message['role'] == 'context' %}
<|im_start|>context
{{ message['content'] }}<|im_end|>
{% elif message['role'] == 'user' %}
<|im_start|>user
{{ message['content'] }}<|im_end|>
{% elif message['role'] == 'assistant' %}
<|im_start|>assistant
{{ message['content'] }}<|im_end|>
{% endif %}
{% endfor %}"""

In [ ]:
from unsloth.chat_templates import get_chat_template
from datasets import load_dataset

prompt_role_system = """Eres Aerito, un asistente de IA especializado en temas de matrícula, procedimientos y trámites académicos de la Facultad de Ciencias de la Universidad Nacional de Ingeniería del Perú.
Deberás responder a los mensajes asegurándote de cumplir con los siguientes criterios.
    1. Proporcionar respuestas informativas, útiles y concisas a las preguntas del usuario basándote exclusivamente en el contexto proporcionado.
    2. Mantén un tono cordial, empático y servicial en sus interacciones.
    3. Preferiblemente, evita derivar o sugerir el contacto con una oficina a menos que sea necesario. Si no hay otra oficina más idónea, la derivación se realizará hacia la Oficina de Estadística (AERA) de la Facultad de Ciencias.
    4. En caso de no tener conocimiento sobre lo consultado, expresa con empatía que no tienes acceso a dicha información."""

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5"
)

def formatting_prompts_func(examples):
    convos = examples["turns"]
    for convo in convos:
        if convo[0]["role"] != "system":
            convo.insert(0, {
                "role": "system",
                "content": prompt_role_system
            })

            convo.insert(0, {
                "role": "system",
                "content": prompt_role_system
            })
            
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

dataset = load_dataset("json", data_files = {"train":"./conversational_faq/data/train_turns_dataset.json", "val": "./conversational_faq/data/val_turns_dataset.json"})
dataset = dataset.map(formatting_prompts_func, batched=True)
print(dataset['train'][0]['text'])

<|im_start|>system
Eres Aerito, un asistente de IA especializado en temas de matrícula, procedimientos y trámites académicos de la Facultad de Ciencias de la Universidad Nacional de Ingeniería del Perú.
Deberás responder a los mensajes asegurándote de cumplir con los siguientes criterios.
    1. Proporcionar respuestas informativas, útiles y concisas a las preguntas del usuario basándote exclusivamente en la información relacionada con la Facultad de Ciencias que conoces.
    2. Mantén un tono cordial, empático y servicial en sus interacciones.
    3. Preferiblemente, evita derivar o sugerir el contacto con una oficina a menos que sea necesario. Si no hay otra oficina más idónea, la derivación se realizará hacia la Oficina de Estadística (AERA) de la Facultad de Ciencias.
    4. En caso de no tener conocimiento sobre lo consultado, expresa con empatía que no tienes acceso a dicha información.<|im_end|>
<|im_start|>user
Si un alumno no cumplió con el pago del autoseguro, ¿qué procedimie

In [39]:
from unsloth import is_bf16_supported
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    eval_dataset = dataset['val'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps=5,
        num_train_epochs=2,
        #max_steps=60,
        learning_rate= 2e-4,
        fp16= not is_bf16_supported(),
        bf16= is_bf16_supported(),
        logging_steps=1,
        eval_strategy= "epoch",
        save_strategy= "epoch",
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs"
    ),
)



Unsloth: Tokenizing ["text"]: 100%|██████████| 5821/5821 [00:07<00:00, 756.38 examples/s]


In [40]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part= "<|im_start|>user\n",
    response_part= "<|im_start|>assistant\n",
)

Map (num_proc=96): 100%|██████████| 1919/1919 [00:00<00:00, 2732.28 examples/s]


In [41]:
print(tokenizer.decode(trainer.train_dataset[5]["input_ids"]))

<|im_start|>system
Eres Aerito, un asistente de IA especializado en temas de matrícula, procedimientos y trámites académicos de la Facultad de Ciencias de la Universidad Nacional de Ingeniería del Perú.
Deberás responder a los mensajes asegurándote de cumplir con los siguientes criterios.
    1. Proporcionar respuestas informativas, útiles y concisas a las preguntas del usuario basándote exclusivamente en la información relacionada con la Facultad de Ciencias que conoces.
    2. Mantén un tono cordial, empático y servicial en sus interacciones.
    3. Preferiblemente, evita derivar o sugerir el contacto con una oficina a menos que sea necesario. Si no hay otra oficina más idónea, la derivación se realizará hacia la Oficina de Estadística (AERA) de la Facultad de Ciencias.
    4. En caso de no tener conocimiento sobre lo consultado, expresa con empatía que no tienes acceso a dicha información.<|im_end|>
<|im_start|>user
¿Qué métodos de pago pueden usar los estudiantes para su orden de p

In [42]:
space = tokenizer(" ", add_special_tokens= False).input_ids[0]
print(tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[0]["labels"]]))

                                                                                                                                                                                                                                                                                         Si un alumno no cumplió con el pago del autoseguro, no podrá realizar la matrícula rezagada. Para gestionar su matrícula rezagada, primero debe realizar el pago del autoseguro médico estudiantil. Luego, deberá seguir estos pasos:

1. Generar la orden de pago por concepto de matrícula rezagada a través del portal INTRALU y realizar el pago correspondiente de S/ 26 en una agencia del BCP o mediante su banca móvil.
2. Presentarse en la Oficina de Estadística (AERA) en las fechas establecidas en el calendario de actividades académicas, llevando el comprobante de pago.

Recuerde que la matrícula rezagada se realiza solo si se cumplen todos los requisitos y en las fechas establecidas.<|im_end|>



In [43]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA RTX 4500 Ada Generation. Max memory = 23.994 GB.
1.77 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,821 | Num Epochs = 2 | Total steps = 728
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 35,192,832/500,000,000 (7.04% trained)


Epoch,Training Loss,Validation Loss


In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
)


FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": "como me matriculo este ciclo"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs , max_new_tokens = 64, use_cache = True, temperature = 1.5, min_p = 0.1)
print(tokenizer.batch_decode(outputs)[0])

The following generation flags are not valid and may be ignored: ['temperature', 'min_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Continue the fibonacci sequence: 1, 1, 2, 3, 5, 8,<|im_end|>
<|im_start|>assistant
The Fibonacci sequence is a series of numbers in which each number is the sum of the two preceding ones. The sequence starts with 1 and 1, and each subsequent number is the sum of the two preceding ones. The sequence goes like this: 1, 1, 2, 3, 5
